# i Process observed data

Once we start simulating outcomes, it will be helpful to have real-world data to compare them to. In this example, we are looking at the first wave of the pandemic for the area of Maryland, DC, and northern Virginia. This script processes data on observed cases and deaths from the [Johns Hopkins University](https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_time_series)for the FIPs codes (counties) represented in our GREASYPOP population.

**Input files**
- `people_all.csv`
- `counties.csv`
- `time_series_covid19_confirmed_US.csv`
- `time_series_covid19_deaths_US.csv`

**Output files**
- `observed_by_county.csv`
- `observed.csv`


In [1]:
# Import packages and set path
import covasim as cv
import numpy as np
from scipy import optimize
import os
import pandas as pd
import matplotlib.pyplot as plt

path  = ""

Covasim 3.1.6 (2024-01-28) — © 2020-2024 by IDM


### i.1 Identify counties in GREASYPOP
Identify all the counties represented in your GREASYPOP population and put them in a dataframe. For this example, the list of counties includes Maryland, DC, and counties in northern Virginia. The county with value 0 is for all the people who commute in to the GREASYPOP area; they will be filter out later.

In [2]:
# Identify all the counties represented in your synthetic population
people = pd.read_csv(f'{path}/people_all.csv',low_memory=False)
counties = people['county'].unique()
counties = pd.DataFrame(counties, columns=['county'])
counties['county'] = counties['county'].astype(str)
counties = counties.set_index('county')
counties

""
county
24031
24037
24013
11001
24510
24027
24033
51013
51059


### i.2 Case data
Process case data from the file `time_series_covid19_confirmed_US.csv`.

In [3]:
counties = pd.read_csv(f'{path}/counties.csv') # Read in the counties represented in the synth pop
counties['county'] = counties['county'].astype(str) # Make county column a string
cases_jhu = pd.read_csv(f'{path}/time_series_covid19_confirmed_US.csv') # Read in case data
cases = cases_jhu[cases_jhu['Province_State'].isin(['Maryland', 'Virginia', 'District of Columbia'])] # Select relevant states
cases = cases.drop(columns=['UID', 'iso2', 'iso3', 'code3', 'Admin2', 'Country_Region', # Drop unneeded columns
                            'Lat', 'Long_', 'Combined_Key','Province_State'])
cases['FIPS'] = cases['FIPS'].astype(int).astype(str) # Make the FIPS code a string
cases.rename(columns={'FIPS':'county'}, inplace=True) # Rename FIPS to county
cases = counties.merge(cases, how='left',on='county') # Merge the cases data to the counties in your synth pop
cases = cases.fillna(0) # Fill in nulls with 0
cases = cases.loc[cases['county'] != '0'] # Get rid of county 0 (this is people who live outside the synth pop)
cases = cases.set_index('county') # Set 'county as the index
cases = cases.T # Pivit the table to be in long format
cases['cum_infections'] = cases.sum(axis=1) # Sum across all counties
cases = cases.reset_index() # Reset index
cases = cases[['index', 'cum_infections']] # Keep needed columns
cases.rename(columns={'index':'date'}, inplace=True) # Rename 'index' column as 'date'
cases.set_index('date', inplace=True) # Set 'date' as the index
cases # Preview dataframe

county,cum_infections
date,
1/22/20,0.0
1/23/20,0.0
1/24/20,0.0
1/25/20,0.0
1/26/20,0.0
...,...
3/5/23,2228591.0
3/6/23,2229116.0
3/7/23,2231126.0


### i.3 Death data
Process the death data from the file `time_series_covid19_deaths_US.csv`.

In [4]:
deaths_jhu = pd.read_csv(f'{path}/time_series_covid19_deaths_US.csv') # Read in file
deaths = deaths_jhu[deaths_jhu['Province_State'].isin(['Maryland', 'Virginia', 'District of Columbia'])] # Keep relevant states
deaths = deaths.drop(columns=['UID', 'iso2', 'iso3', 'code3', 'Admin2', 'Country_Region', # Drop unneeded columns
                              'Lat', 'Long_', 'Combined_Key','Province_State',])
deaths['FIPS'] = deaths['FIPS'].astype(int).astype(str) # Make the FIPS code a string
deaths.rename(columns={'FIPS':'county'}, inplace=True) # Rename the 'FIPS' column as 'county'
deaths = counties.merge(deaths, how='left',on='county') # Merge death data to counties in your synth pop
deaths = deaths.fillna(0) # Fill in nulls

### i.4 Deaths per 100k by county for Maryland
Make a subset `md_deaths_per100k` that is deaths per 100k by county and save.

In [5]:
md_counties=['24001','24003','24005','24009','24011',
             '24013','24015','24017','24019','24021',
             '24023','24025','24027','24029','24031',
             '24033','24035','24037','24039','24041',
             '24043','24045','24047','24510']
md_deaths = deaths.loc[deaths['county'].isin(md_counties)]
md_deaths_per100k = md_deaths.copy()
md_deaths_per100k.iloc[:, 2:] = md_deaths.iloc[:, 2:].div(md_deaths['Population'], axis=0) * 100000
md_deaths_per100k = md_deaths_per100k.drop(columns=['Population']).set_index('county')
md_deaths_per100k = md_deaths_per100k.T
for col in md_deaths_per100k.columns:
    md_deaths_per100k[col+'_new_per100k'] = md_deaths_per100k[col].diff()
    md_deaths_per100k = md_deaths_per100k.rename(columns={str(col):str(col)+'_cum_per100k'})
md_deaths_per100k = md_deaths_per100k.reset_index()
md_deaths_per100k = md_deaths_per100k.iloc[10:161,:].reset_index()
md_deaths_per100k['day'] = md_deaths_per100k.index
md_deaths_per100k.to_csv(f'{path}/observed_by_county.csv')

county,level_0,index,24031_cum_per100k,24037_cum_per100k,24013_cum_per100k,24510_cum_per100k,24027_cum_per100k,24033_cum_per100k,24017_cum_per100k,24021_cum_per100k,...,24015_new_per100k,24045_new_per100k,24023_new_per100k,24019_new_per100k,24011_new_per100k,24039_new_per100k,24041_new_per100k,24035_new_per100k,24029_new_per100k,day
0,10,2/1/20,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0
1,11,2/2/20,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,1
2,12,2/3/20,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,2
3,13,2/4/20,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,3
4,14,2/5/20,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146,156,6/26/20,69.478285,44.048982,65.302439,55.603296,25.791397,74.340694,52.67768,44.693254,...,0.0,0.000000,-3.446612,0.0,0.0,0.0,0.0,1.984875,0.0,146
147,157,6/27/20,69.858988,44.929962,65.302439,56.108780,26.098437,74.450665,52.67768,45.078541,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,147
148,158,6/28/20,69.954163,44.929962,65.896098,56.445770,26.098437,74.560637,52.67768,45.078541,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,148
149,159,6/29/20,70.049339,44.929962,66.489756,56.445770,26.098437,74.780579,52.67768,45.078541,...,0.0,0.965167,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,149


### i.5 Death data continued
Continue processing `deaths` to get cumulative values for all counties in the GREASYPOP.

In [6]:
obs_pop = deaths['Population'].sum() # Create variable that is population summed over all counties
deaths.drop(columns=['Population'], inplace=True) # Drop 'Population' column
#deaths = deaths.loc[cases['county'] != '0']
deaths = deaths.set_index('county') # Set index as 'county'
deaths = deaths.T # Pivot the table in long format
deaths['cum_deaths'] = deaths.sum(axis=1) # Calculate the total deaths over all counties
deaths = deaths.reset_index() # Reset the index
deaths = deaths[['index', 'cum_deaths']] # Keep needed columns
deaths.rename(columns={'index':'date'}, inplace=True) # Rename column 'index' to be 'date'
deaths.set_index('date', inplace=True) # Make column 'date' the index
deaths # Preview data

county,cum_deaths
date,
1/22/20,0.0
1/23/20,0.0
1/24/20,0.0
1/25/20,0.0
1/26/20,0.0
...,...
3/5/23,22106.0
3/6/23,22113.0
3/7/23,22132.0


### i.6 Merged cases and deaths
Now we need to rename some columns so that Covasim recognizes them. See [Covasim Tutorial 4 People and data](https://docs.idmod.org/projects/covasim/en/latest/tutorials/tut_people.html) for more on data requirements. Here are the variables we need:

- `cum_diagnoses`: cumulative confirmed cases
- `new_diagnoses`: daily confirmed cases
- `cum_infections`: cumulative cases (confirmed and otherwise)
- `new_infections`: daily cases (confirmed and otherwise)
- `cum_deaths`: cumulative deaths
- `new_deaths`: daily deaths

Covasim uses the term 'diagnoses' to mean PCR confirmed cases and 'infections' for any case, diagnosed or not. Johns Hopkins uses the term 'infections' for confirmed cases so we need to rename this for Covasim. Also, we are looking at the first wave of the pandemic (2020-02-01 to 2020-06-30) so there are likely many cases that were not confirmed by test; therefore, we multiply the values from Johns Hopkins by 50 so they are closer to the likely total number of cases (diagnosed and undiagnosed).

In [7]:
observed = cases.merge(deaths, how='left', on='date').reset_index() # Merge cases and deaths
observed['cum_diagnoses'] = observed['cum_infections'] # Create new column 'cum_diagnoses' from 'cum_infections'
observed['new_diagnoses'] = observed['cum_diagnoses'].diff() # Create column for daily diagnoses
observed['cum_infections'] = observed['cum_diagnoses'] * 50 # Multiply by 50 so infections more closely reflects cases in the community
observed['new_infections'] = observed['cum_infections'].diff() # # Create column for daily infections
observed['new_deaths'] = observed['cum_deaths'].diff() # Create column for daily deaths
observed['new_infections'] = observed['new_infections'].rolling(window=14).mean() # Create 14-day rolling average for infections
observed['new_deaths'] = observed['new_deaths'].rolling(window=14).mean() # Create 14-day rolling average for deaths
observed = observed[10:161] # Select relevant time frame. First wave 2020-02-01 to 2020-06-30
observed = observed.fillna(0) # Fill nulls
observed['date'] = pd.to_datetime(observed['date'], infer_datetime_format=True) # Reformat date
observed.reset_index(drop=True, inplace=True) # Reset index
observed = observed.set_index('date') # Set the index to be 'date'
observed.to_csv(f'{path}/observed.csv') # Export dataframe
observed # Preview data

/var/folders/9f/7594l8nn5czfjw4cd7xxcw640000gq/T/ipykernel_94128/1396653536.py:11: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  observed['date'] = pd.to_datetime(observed['date'], infer_datetime_format=True) # Reformat date
/var/folders/9f/7594l8nn5czfjw4cd7xxcw640000gq/T/ipykernel_94128/1396653536.py:11: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  observed['date'] = pd.to_datetime(observed['date'], infer_datetime_format=True) # Reformat date


county,cum_infections,cum_deaths,cum_diagnoses,new_diagnoses,new_infections,new_deaths
date,,,,,,
2020-02-01,0.0,0.0,0.0,0.0,0.000000,0.000000
2020-02-02,0.0,0.0,0.0,0.0,0.000000,0.000000
2020-02-03,0.0,0.0,0.0,0.0,0.000000,0.000000
2020-02-04,0.0,0.0,0.0,0.0,0.000000,0.000000
2020-02-05,0.0,0.0,0.0,0.0,0.000000,0.000000
...,...,...,...,...,...,...
2020-06-26,5457200.0,4583.0,109144.0,556.0,30885.714286,27.857143
2020-06-27,5488350.0,4610.0,109767.0,623.0,29560.714286,27.357143
2020-06-28,5515100.0,4626.0,110302.0,535.0,28835.714286,26.785714
